# 🔧 多GPU训练问题诊断与解决

**问题描述**: 在运行多GPU训练时遇到"Terminated"错误，训练进程被意外终止。

**可能原因分析**:
1. **端口冲突**: 多个进程尝试使用同一端口
2. **NCCL通信问题**: GPU间通信配置错误
3. **进程清理过度**: 清理脚本误杀了正在启动的进程
4. **显存不足**: GPU显存不够导致进程崩溃
5. **环境配置问题**: CUDA/NCCL环境变量设置不当

本notebook将系统性地诊断和解决这些问题。

In [ ]:
# 1. 导入必要的调试库
import torch
import torch.distributed as dist
import torch.multiprocessing as mp
import os
import subprocess
import socket
import time
import sys
import psutil
import signal
from datetime import datetime

print(f"PyTorch版本: {torch.__version__}")
print(f"Python版本: {sys.version}")
print(f"当前时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
# 2. 检查GPU可用性和状态
print("=== GPU环境检查 ===")

# 基本CUDA检查
print(f"CUDA可用: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA版本: {torch.version.cuda}")
    print(f"cuDNN版本: {torch.backends.cudnn.version()}")
    print(f"GPU数量: {torch.cuda.device_count()}")
    
    # 详细GPU信息
    for i in range(torch.cuda.device_count()):
        props = torch.cuda.get_device_properties(i)
        print(f"\nGPU {i}: {props.name}")
        print(f"  总显存: {props.total_memory / 1024**3:.1f} GB")
        print(f"  计算能力: {props.major}.{props.minor}")
        
        # 尝试获取显存使用情况
        try:
            torch.cuda.set_device(i)
            allocated = torch.cuda.memory_allocated(i) / 1024**3
            reserved = torch.cuda.memory_reserved(i) / 1024**3
            print(f"  已分配显存: {allocated:.1f} GB")
            print(f"  已保留显存: {reserved:.1f} GB")
        except Exception as e:
            print(f"  显存信息获取失败: {e}")
else:
    print("❌ CUDA不可用，无法进行GPU训练")

In [ ]:
# 3. 验证NCCL后端配置
print("\n=== NCCL后端检查 ===")

# 检查NCCL可用性
try:
    nccl_available = torch.distributed.is_nccl_available()
    print(f"NCCL后端可用: {nccl_available}")
    
    if nccl_available:
        # 检查NCCL版本
        try:
            import subprocess
            result = subprocess.run(['nvidia-smi', '--query-gpu=driver_version', '--format=csv,noheader,nounits'], 
                                  capture_output=True, text=True)
            if result.returncode == 0:
                driver_version = result.stdout.strip().split('\n')[0]
                print(f"NVIDIA驱动版本: {driver_version}")
        except Exception as e:
            print(f"驱动版本检查失败: {e}")
        
        # 检查重要的NCCL环境变量
        nccl_vars = ['NCCL_DEBUG', 'NCCL_TIMEOUT', 'NCCL_IB_DISABLE', 'NCCL_P2P_DISABLE']
        print("\n当前NCCL环境变量:")
        for var in nccl_vars:
            value = os.environ.get(var, '未设置')
            print(f"  {var}: {value}")
    else:
        print("❌ NCCL后端不可用")
        
except Exception as e:
    print(f"NCCL检查失败: {e}")

In [ ]:
# 4. 测试分布式设置和端口可用性
print("\n=== 分布式设置测试 ===")

def find_free_port(start_port=12355, end_port=12370):
    """查找可用端口"""
    for port in range(start_port, end_port):
        try:
            with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
                s.bind(('localhost', port))
                return port
        except OSError:
            continue
    return None

# 查找可用端口
print("查找可用端口...")
for port in range(12355, 12370):
    try:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.bind(('localhost', port))
            print(f"✅ 端口 {port}: 可用")
            free_port = port
            break
    except OSError:
        print(f"❌ 端口 {port}: 被占用")
else:
    free_port = None
    print("⚠️  在12355-12369范围内未找到可用端口")

if free_port:
    print(f"\n推荐使用端口: {free_port}")

# 检查网络连接
print("\n检查localhost连接...")
try:
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.settimeout(1)
        result = s.connect_ex(('localhost', 22))  # 尝试连接SSH端口
        if result == 0:
            print("✅ localhost网络连接正常")
        else:
            print("⚠️  localhost连接可能有问题")
except Exception as e:
    print(f"⚠️  网络检查异常: {e}")

In [ ]:
# 5. 调试CUDA进程和资源占用
print("\n=== CUDA进程检查 ===")

def check_cuda_processes():
    """检查CUDA相关进程"""
    try:
        # 使用nvidia-smi检查GPU进程
        result = subprocess.run(['nvidia-smi', 'pmon', '-c', '1'], 
                              capture_output=True, text=True, timeout=10)
        if result.returncode == 0:
            print("当前GPU进程:")
            print(result.stdout)
        else:
            print("nvidia-smi检查失败")
    except subprocess.TimeoutExpired:
        print("nvidia-smi命令超时")
    except Exception as e:
        print(f"GPU进程检查失败: {e}")
    
    try:
        # 检查Python训练进程
        result = subprocess.run(['ps', 'aux'], capture_output=True, text=True, timeout=5)
        if result.returncode == 0:
            python_procs = []
            for line in result.stdout.split('\n'):
                if 'python' in line and any(keyword in line for keyword in ['train', 'ddp', 'main_train']):
                    python_procs.append(line.strip())
            
            if python_procs:
                print("\n发现Python训练进程:")
                for proc in python_procs:
                    print(f"  {proc}")
            else:
                print("\n✅ 未发现冲突的Python训练进程")
    except Exception as e:
        print(f"进程检查失败: {e}")

check_cuda_processes()

# 检查core dump文件
print("\n检查core dump文件...")
try:
    import glob
    core_files = glob.glob('/work/home/luoyinze/paligemma-VLA/paligemma-VLA/core.*')
    if core_files:
        print(f"发现 {len(core_files)} 个core dump文件:")
        for core_file in core_files[-3:]:  # 只显示最近的3个
            stat = os.stat(core_file)
            size_mb = stat.st_size / 1024 / 1024
            mtime = datetime.fromtimestamp(stat.st_mtime)
            print(f"  {os.path.basename(core_file)}: {size_mb:.1f}MB, {mtime}")
        print("\n⚠️  存在core dump文件表明之前有进程崩溃")
    else:
        print("✅ 未发现core dump文件")
except Exception as e:
    print(f"core dump检查失败: {e}")

In [ ]:
# 6. 修复建议和安全的训练启动方法
print("\n=== 问题修复建议 ===")

# 分析问题
print("🔍 问题分析:")
print("1. 'Terminated'错误通常是由于进程被意外终止")
print("2. 可能的原因包括:")
print("   - 进程清理脚本过于激进，误杀了正在启动的进程")
print("   - 端口冲突导致初始化失败")
print("   - NCCL通信配置问题")
print("   - 显存不足或资源竞争")

print("\n🛠️  修复方案:")
print("1. 移除进程清理代码中的过度清理")
print("2. 改进端口检测和分配机制")
print("3. 添加更好的错误处理和同步")
print("4. 使用更安全的启动方式")

# 提供修复后的启动命令
print("\n🚀 建议的安全启动方式:")
print("")
print("方案1: 使用修复后的脚本 (推荐)")
print("python main_train_ddp.py \\")
print("    --config_path configs/vla_config_ddp.yaml \\")
print("    --experiment_name test_fixed \\")
print("    --batch_size 32 \\")
print("    --epochs 2")
print("")
print("方案2: 使用环境变量控制")
print("NCCL_DEBUG=WARN NCCL_TIMEOUT=1800 \\")
print("python main_train_ddp.py \\")
print("    --config_path configs/vla_config_ddp.yaml \\")
print("    --experiment_name test_env \\")
print("    --batch_size 16 \\")
print("    --epochs 1")
print("")
print("方案3: 先测试2个GPU")
print("CUDA_VISIBLE_DEVICES=0,1 \\")
print("python main_train_ddp.py \\")
print("    --config_path configs/vla_config_ddp.yaml \\")
print("    --experiment_name test_2gpu \\")
print("    --batch_size 16 \\")
print("    --epochs 1")

In [ ]:
# 7. 执行实际修复 - 清理环境并准备安全启动
print("\n=== 执行环境清理和修复 ===")

# 安全清理core dump文件
print("1. 清理core dump文件...")
try:
    import glob
    import os
    core_files = glob.glob('/work/home/luoyinze/paligemma-VLA/paligemma-VLA/core.*')
    if core_files:
        for core_file in core_files:
            try:
                os.remove(core_file)
                print(f"   删除: {os.path.basename(core_file)}")
            except Exception as e:
                print(f"   删除失败 {os.path.basename(core_file)}: {e}")
        print(f"✅ 清理了 {len(core_files)} 个core dump文件")
    else:
        print("✅ 无需清理core dump文件")
except Exception as e:
    print(f"⚠️  core dump清理失败: {e}")

# 设置安全的NCCL环境变量
print("\n2. 设置NCCL环境变量...")
safe_nccl_env = {
    'NCCL_DEBUG': 'WARN',
    'NCCL_TIMEOUT': '1800',
    'NCCL_SOCKET_TIMEOUT': '600', 
    'NCCL_IB_DISABLE': '1',
    'NCCL_P2P_DISABLE': '1'
}

for key, value in safe_nccl_env.items():
    os.environ[key] = value
    print(f"   设置 {key} = {value}")

print("✅ NCCL环境变量配置完成")

# 检查修复后的脚本是否存在
print("\n3. 检查训练脚本...")
script_path = '/work/home/luoyinze/paligemma-VLA/paligemma-VLA/main_train_ddp.py'
if os.path.exists(script_path):
    print(f"✅ 找到训练脚本: {script_path}")
    
    # 简单检查脚本内容
    with open(script_path, 'r') as f:
        content = f.read()
        if 'pkill' in content:
            print("⚠️  脚本中仍包含pkill命令，建议移除")
        else:
            print("✅ 脚本已移除危险的进程清理命令")
else:
    print(f"❌ 未找到训练脚本: {script_path}")

print("\n✅ 环境修复完成！现在可以安全启动训练")

## 🎯 最终测试和启动指南

根据上述诊断结果，现在可以安全地启动多GPU训练了。

### 推荐的启动顺序:

1. **首先测试小规模训练** (2个GPU, 小batch size)
2. **然后扩展到全部GPU**
3. **最后增加batch size到目标值**

### 如果仍然遇到问题:

1. **检查显存**: 确保每个GPU有足够显存
2. **减少batch size**: 从16开始测试
3. **检查数据路径**: 确保训练数据可访问
4. **查看详细日志**: 启用NCCL_DEBUG=INFO获取更多信息

### 监控命令:

在另一个终端运行以下命令监控训练:
```bash
# 监控GPU使用
watch -n 1 nvidia-smi

# 监控训练日志
tail -f experiments/test_fixed/logs/train_log_ddp_*.log
```